<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Cómo usar la API de FourSquare con Python</font></h1>


## Introducción

En este laboratorio usted aprenderá detalladamente a realizar llamadas a la API de FourSquare con distintos propositos. Aprenderá a construir una URL para enviar peticiones a la API para buscar lugares específicos, explorar un sitio en particular, un usuario de FourSquare o alguna referencia geografica, además, podrá obtener los lugares de moda alrededor de alguna locación. También aprederá a usar la librería de visualización Folium para ver los resultados.


## Indice

1.  <a href="https://#item1">Función de Busqueda de la API de Foursquare</a>
2.  <a href="https://#item2">Explorar un Lugar</a>
3.  <a href="https://#item3">Explorar un Usuario</a>
4.  <a href="https://#item4">Función de Exploración de la API de Foursquare</a>
5.  <a href="https://#item5">Obtener Lugares de Moda</a>


### Importar las librerías necesarias


In [ ]:
import requests # librería para manejar las solicitudes
import pandas as pd # librería para análisis de datos
import numpy as np # librería para manejar datos vectorizados
import random # librería para generar números aleatorios


!pip install geopy
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 

# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# librería para convertir un archivo json en un dataframe pandas
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # librería para graficar 

print('Folium installed')
print('Libraries imported.')

### Definir la Versión y Credenciales de FourSquare


##### Asegúrese de haber creado una cuenta de desarrollador en FourSquare con las credenciales apropiadas


In [ ]:
CLIENT_ID = '' # su ID de Foursquare
CLIENT_SECRET = '' # su Secreto de Cliente de Foursquare
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Vamos a suponer que usted esta hospedado en el hotel Conrad. Empecemos por convertir la dirección del hotel en valores de coordenadas dadas por la latitud y la longitud.


Para definir una instancia del geocoder necesitamos definir un user_agent. Nombraremos a nuestro agente <em>foursquare_agent</em> de la siguiente forma.


In [ ]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

<a id="item1"></a>


## 1. Búsqueda por categoría de un lugar específico

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### Supongamos que es la hora de comer y usted desea comida italiana. Vamos a definir una consulta para buscar comida italiana dentro de un radio de 500 metros del hotel Conrad.


In [ ]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

#### Definir la URL correspondiente


In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

#### Enviar la petición GET y examinar los resultados


In [ ]:
results = requests.get(url).json()
results

#### Obtener la parte relevante del JSON y convertirlo en un dataframe *pandas*


In [ ]:
# asignar la parte relevante del JSON a la variable venues
venues = results['response']['venues']

# convertir venues en un dataframe
dataframe = json_normalize(venues)
dataframe.head()

#### Definir la información de interes y filtrar el dataframe


In [ ]:
# mantener unicamente las columnas que incluyan el nombre del lugar y cualquier cosa asociada a la ubicación
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# función para extraer la categoría del lugar
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtrar la categoría para cada fila
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# limpiar la columna "names" manteniendo solo el último término
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

#### Visualicemos los restaurantes italianos cercanos


In [ ]:
dataframe_filtered.name

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# añadir un circulo rojo que represente el Hotel Conrad
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# añadir los restaurantes italianos como circulo azules
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# mostrar el mapa
venues_map

<a id="item2"></a>


## 2. Explorar un Lugar Dado

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Investiguemos el restaurante italiano mas cercano -- *Harry's Italian Pizza Bar*


In [ ]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

#### Enviar la petición GET y examinar los resultados


In [ ]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

### B. Obtener la puntuación general del lugar


In [ ]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

No es una puntuación muy buena. Revisemos la puntuación del siguiente restaurante italiano mas cercano.


In [ ]:
venue_id = '4f3232e219836c91c7bfde94' # ID del Restaurante Conca Cucina Italian
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Debido a que este restaurante no tiene puntuación, revisemos un tercero


In [ ]:
venue_id = '3fd66200f964a520f4e41ee3' # ID de Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Este restaurante tiene un puntuación ligeramente mejor, revisemosla con mas detalle.


### C. Obtener el número de consejos


In [ ]:
result['response']['venue']['tips']['count']

### D. Obtener los consejos de un lugar

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**


#### Generar una URL y enviar una solicitud GET. Asegúrese de establecer el límite para obtener todos los consejos


In [ ]:
## Consejos de Ecco
limit = 15 # establece el límite para ser mayor o igual al número total de consejos
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

#### Obtener consejos y la lista de características asociadas


In [ ]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

#### Dar formato al ancho de la columna y mostrar todos los consejos


In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # consejos json normalizados

# columnas a mantener
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# mostrar consejos
tips_filtered.reindex()

Recuerde que debido a que estamos usando una cuenta personal de desarrollador solo podemos acceder a 2 de los 15 consejos del restaurante.


<a id="item3"></a>


## 3. Buscar a un Usuario de FourSquare

> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### Definir la URL, enviar la solicitud GET y mostrar las características asociadas con el usuario


In [ ]:
idnumber = '48454224' # ID del usuario con mayor número de aceptación y perfil completo 

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# enviar solicitud GET
results = requests.get(url).json()
#user_data = results['response']['user']
user_data=results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']#['items']
# mostrar características asociadas con el usuario
#user_data.keys()
#results
pd.set_option('display.max_colwidth', -1)

users_df = json_normalize(user_data)

# filtrar columnas
filtered_columns = ['id','user.id','user.firstName','user.lastName']
tips_filtered = users_df.loc[:, filtered_columns]

# mostrar consejos del usuario
df=tips_filtered
df
#url

In [ ]:

g=df.loc[df['user.id'] == '133773133']
print('First Name: ' + g['user.firstName'])
print('Last Name: ' + g['user.lastName'])


### Obtener la Imagen del Perfil del Usuario


In [ ]:
# 1. tomar el prefijo de la foto
# 2. tomar el sufijo de la foto
# 3. concatenarlos utilizando el tamaño de la imagen
Image(url='https://fastly.4sqi.net/img/general/540x920/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg')

¡Wow! Parece ser que Nick es un usuario muy activo en FourSquare con mas de 250 consejos.


### Obtener las Recomendaciones del usuario


In [ ]:
# definir la URL de las recomendaciones
user_id='484542633'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# enviar la solicitud GET y obtener las recomendaciones del usuario
results = requests.get(url).json()
tips = results['response']['tips']['items']

# dar formato al ancho de la columna
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filtrar columnas
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# mostrar las recomendaciones del usuario
tips_filtered

#### Obtengamos el lugar para la recomendación con el mayor número de aceptaciones


In [ ]:
tip_id = '5ab5575d73fe2516ad8f363b' # id de la recomendación

# definir la URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# enviar la solicitud GET y examinar los resultados
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

## 4. Explorar una Ubicación

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### Ha definido su platillo gourmet en Ecco y siente curiosidad acerca de los sitios populares alrededor del restaurante. Para explorar el área, empecemos por obtener los valores de latitud y la longitud del Restaurant Ecco.


In [ ]:
latitude = 40.715337
longitude = -74.008848

#### Definir una URL


In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

#### Enviar una solicitud GET y examinar los resultados


In [ ]:
import requests

In [ ]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

#### Obtener la parte relevante del objeto JSON


In [ ]:
items = results['response']['groups'][0]['items']
items[0]

#### Procesar el objeto JSON, convetirlo y limpiar el dataframe


In [ ]:
dataframe = json_normalize(items) # JSON

# filtrar columnas
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filtrar la categoría para cada columna
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# limpiar columnas
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

#### Visualicemos estos elementos en el mapa alrededor de nuestra ubicación


In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# Añadir Ecco como una marca circular roja
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# añadir sitios populares al mapa como marcas circulares azules
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# mostrar mapa
venues_map

<a id="item5"></a>


## 5. Explorar Sitios Populares

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Ahora, en vez de explorar  el área alrededor de Ecco, es de interes saber los sitios que son populares durante su comida, esto es, los lugares con el mayor tráfico peatonal. Hagamoslo y obtengamos los sitios populares alrededor de Ecco.


In [ ]:
# definir la URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# enviar la solicitud GET y obtener los sitios de moda
results = requests.get(url).json()
results

### Revisar si algún sitio esta de moda en este momento


In [ ]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filtrar columnas
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filtrar la categoría para cada fila
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [ ]:
# mostrar los sitios populares
trending_venues_df

Ahora, dependiendo de cuando usted ejecute el codigo anterior, podría obtener distintos sitios ya que la información de tráfico peatonal se obtiene en tiempo real.


### Visuaizar sitios populares


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # añadir Ecco como una marca circular roja
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # añadir los sitios populares como marcas circulares azules
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# mostrar el mapa
venues_map

<a id="item6"></a>


### Gracias por terminar este laboratorio

Este cuaderno fue creado por [Alex Aklson](https://www.linkedin.com/in/aklson?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Espero que haya encontrado este laboratorio de su interes y educativo. Tómese la libertad de contactar conmigo para cualquier duda o aclaración.

Este cuaderno fue modificado por Nayef Abou Tayoun ([https://www.linkedin.com/in/nayefaboutayoun/](https://www.linkedin.com/in/nayefaboutayoun?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDS0701ESCoursera21685115-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ))


Este cuaderno forma parte del curso en **Coursera** llamado *Applied Data Science Capstone*. Si accede a este cuaderno desde afuera del curso, puede tomarlo en línea haciendo clic [aquí](http://cocl.us/DP0701EN_Coursera_Week2\_LAB1).


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
